## 安装依赖

In [3]:
!pip install langchain langchain-community pydantic jq faiss-cpu sentence-transformers

## 创建示例数据

In [4]:
# =========================
# 1. 加载 JSON 数据
# =========================
import json

# 这里示范数据结构，实际可替换成你的 chat_history.json
json_data = [
    {
        "id": 1,
        "speaker": "用户A",
        "content": "昨天我们讨论的那个项目截止日期定在什么时候？我记得是下周。",
        "timestamp": "2025-05-01T10:00:00"
    },
    {
        "id": 2,
        "speaker": "用户B",
        "content": "是的，最初定的是下周五。但现在项目需求有所增加，我们最好推迟到后下周三。",
        "timestamp": "2025-05-01T10:05:00"
    },
    {
        "id": 3,
        "speaker": "用户A",
        "content": "后下周三？好的，这个时间我可以接受。我会在会议记录里更新。",
        "timestamp": "2025-05-01T10:10:00"
    }
]


## Langchain加载数据、转换为doc对象

In [5]:
# =========================
# 2. LangChain 转换为 Document
# =========================
from langchain_core.documents import Document

documents = [
    Document(
        page_content=item["content"],
        metadata={
            "speaker": item["speaker"],
            "timestamp": item["timestamp"],
            "id": item["id"]
        }
    )
    for item in json_data
]

print(f"✅ 转换为 Document 完成，共 {len(documents)} 条记录")
print(documents[0].page_content, documents[0].metadata)

✅ 转换为 Document 完成，共 3 条记录
昨天我们讨论的那个项目截止日期定在什么时候？我记得是下周。 {'speaker': '用户A', 'timestamp': '2025-05-01T10:00:00', 'id': 1}


## chunking切分

In [17]:
# =========================
# 3. 文本 Chunking
# =========================
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,      # 每块最大字符数
    chunk_overlap=10,   # 重叠字符数
    separators=["\n\n", "\n", "。", "！", "？", " ", ""]
)

chunks = splitter.split_documents(documents)
print(f"✅ 文本切分完成，总共 {len(chunks)} 个 chunk")
for chunk in chunks:
    print(chunk.page_content, chunk.metadata)
    print(chunk,"\n")


✅ 文本切分完成，总共 3 个 chunk
昨天我们讨论的那个项目截止日期定在什么时候？我记得是下周。 {'speaker': '用户A', 'timestamp': '2025-05-01T10:00:00', 'id': 1}
page_content='昨天我们讨论的那个项目截止日期定在什么时候？我记得是下周。' metadata={'speaker': '用户A', 'timestamp': '2025-05-01T10:00:00', 'id': 1} 

是的，最初定的是下周五。但现在项目需求有所增加，我们最好推迟到后下周三。 {'speaker': '用户B', 'timestamp': '2025-05-01T10:05:00', 'id': 2}
page_content='是的，最初定的是下周五。但现在项目需求有所增加，我们最好推迟到后下周三。' metadata={'speaker': '用户B', 'timestamp': '2025-05-01T10:05:00', 'id': 2} 

后下周三？好的，这个时间我可以接受。我会在会议记录里更新。 {'speaker': '用户A', 'timestamp': '2025-05-01T10:10:00', 'id': 3}
page_content='后下周三？好的，这个时间我可以接受。我会在会议记录里更新。' metadata={'speaker': '用户A', 'timestamp': '2025-05-01T10:10:00', 'id': 3} 



## embedding成为句向量

In [18]:
# =========================
# 4. Embedding
# =========================
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="m3e-small")

# =========================
# 5. 转换成向量
# =========================
# FAISS 需要的就是直接把 Document 列表传入向量数据库，它会自动计算 embedding
# 但如果你想单独拿 vector，也可以：
vectors = [embedding_model.embed_documents([chunk.page_content])[0] for chunk in chunks]


## 存入vectordb

In [19]:
# =========================
# 6. 存入向量数据库（FAISS）
# =========================
from langchain_community.vectorstores import FAISS

vector_db = FAISS.from_documents(chunks, embedding_model)

# 保存到本地
vector_db.save_local("chat_vector_db")
print("✅ 已保存向量数据库到 chat_vector_db/")

✅ 已保存向量数据库到 chat_vector_db/


## 测试topk检索

In [20]:
# =========================
# 7. 测试 top-k 检索
# =========================
query = "时间？"
results = vector_db.similarity_search(query, k=2)
print(results,"\n")
print("🔎 top-k 检索结果：")
for r in results:
    print(f"内容：{r.page_content}")
    print(f"说话者：{r.metadata['speaker']}, 时间：{r.metadata['timestamp']}")
    print("-" * 50)


[Document(id='f89cc412-6525-4faf-864d-11034d756f0b', metadata={'speaker': '用户A', 'timestamp': '2025-05-01T10:00:00', 'id': 1}, page_content='昨天我们讨论的那个项目截止日期定在什么时候？我记得是下周。'), Document(id='4882b0bd-bea8-4ccd-a657-509374da75a3', metadata={'speaker': '用户A', 'timestamp': '2025-05-01T10:10:00', 'id': 3}, page_content='后下周三？好的，这个时间我可以接受。我会在会议记录里更新。')] 

🔎 top-k 检索结果：
内容：昨天我们讨论的那个项目截止日期定在什么时候？我记得是下周。
说话者：用户A, 时间：2025-05-01T10:00:00
--------------------------------------------------
内容：后下周三？好的，这个时间我可以接受。我会在会议记录里更新。
说话者：用户A, 时间：2025-05-01T10:10:00
--------------------------------------------------
